In [55]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content"
# !kaggle competitions download -c tabular-playground-series-aug-2022
# !unzip tabular-playground-series-aug-2022
# for dirname, _, filenames in os.walk('/content'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [56]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [57]:
!pip install feature_engine
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
import pickle
import gc; gc.enable()
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from feature_engine.encoding import WoEEncoder
from sklearn.linear_model import HuberRegressor
import warnings; warnings.filterwarnings("ignore")
from tensorflow.keras import Sequential  
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
pd.options.display.max_columns = 999

In [59]:
train.isna().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [60]:
print(np.mean(train["failure"]))
print(np.max(train["failure"]))
print(np.min(train["failure"]))
print(np.std(train["failure"]))

0.21260820474219044
1
0
0.4091527294525765


In [61]:
test.isna().sum()

id                   0
product_code         0
loading            223
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      329
measurement_4      409
measurement_5      508
measurement_6      624
measurement_7      720
measurement_8      846
measurement_9      904
measurement_10    1067
measurement_11    1136
measurement_12    1240
measurement_13    1303
measurement_14    1440
measurement_15    1542
measurement_16    1678
measurement_17    1740
dtype: int64

In [62]:
print(train['product_code'].value_counts())
print(test['product_code'].value_counts())
train.head()

C    5765
E    5343
B    5250
D    5112
A    5100
Name: product_code, dtype: int64
F    5422
I    5228
G    5107
H    5018
Name: product_code, dtype: int64


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [63]:
print(train.columns)
print(test.columns)

Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17',
       'failure'],
      dtype='object')
Index(['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17'],
      dtype='object')


In [64]:
folds_dict = {'Fold 1': [['C', 'D', 'E'], ['A', 'B']], 
               'Fold 2': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 3': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 4': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 5': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 6': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 7': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 8': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 9': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 10': [['A', 'B', 'C'], ['D', 'E']]}
print(folds_dict["Fold 1"])

[['C', 'D', 'E'], ['A', 'B']]


In [65]:
tf.keras.utils.set_random_seed(42)
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    features = 0
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m4_missing'] = data['measurement_4'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']

    # dictionnary of dictionnaries (for the 11 best correlated measurement columns), 
    # we will use the dictionnaries below to select the best correlated columns according to the product code)
    # Only for 'measurement_17' we make a 'manual' selection :
    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    # print(col)
    a = []
    b =[]

    #sort the measurement as its correlation among measurement_3-17
    corr = np.absolute(data.drop(col, axis=1).corr())
    for x in range(3,17):
        corrr = corr[f'measurement_{x}'].sort_values(ascending=False)
        a.append(np.round(np.sum(corrr[1:4]),3)) # we add the 3 first lines of the correlation values to get the "most correlated"
        b.append(f'measurement_{x}')
    print(a,b)
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)
    print(f'Columns selected by correlation sum of the 3 first rows : ')
    display(c.head(15))

    for i in range(10): 
        measurement_col = c.iloc[i, 0]
        fill_dict = {}
        for x in data.product_code.unique() : 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            measurement_col_dic = {}
            fill_dict[x] = corr[1:5].index.tolist()
        full_fill_dict[measurement_col] = fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f == 'loading']
    print('failure' in feature)
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()]
    for code in data.product_code.unique():
        total_na_filled_by_linear_model = 0
        print(f'\n-------- Product code {code} ----------\n')
        print(f'filled by linear model :')
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any') # a na in row then drop
            '''
            every column should have value && measurement should not
            '''
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]
            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])
            print(f'{measurement_col} : {len(tmp_test)}')
            total_na_filled_by_linear_model += len(tmp_test)
            # break
        # others NA columns:
        # print(data.loc[data["product_code"] == code, nullValue_cols].isnull().sum())
        # print(data.loc[data["product_code"] == code, nullValue_cols].isnull().sum().sum())
        NA = data.loc[data["product_code"] == code, nullValue_cols].isnull().sum().sum()
        '''
        KNNImputer
        '''
        model1 = KNNImputer(n_neighbors=3)
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])
        print(f'\n{total_na_filled_by_linear_model} filled by linear model ') 
        print(f'{NA} filled by KNN ')
        # break
    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    data['measurement_std'] = data[[f'measurement_{i}' for i in range(3, 17)]].std(axis=1)
    data['measurement_median'] = data[[f'measurement_{i}' for i in range(3, 17)]].median(axis=1)
    data['measurement_max'] = data[[f'measurement_{i}' for i in range(3, 17)]].max(axis=1)
    data['measurement_min'] = data[[f'measurement_{i}' for i in range(3, 17)]].min(axis=1)
    data['measurement_skew'] = data[[f'measurement_{i}' for i in range(3, 17)]].skew(axis=1)
    
    
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]

    '''
    drop attribute_1 cuz they have different unique values between train and test
    '''
    woe_encoder = WoEEncoder(variables=['attribute_0'])
    woe_encoder.fit(df_train, df_train['failure'])
    df_train = woe_encoder.transform(df_train)
    df_test = woe_encoder.transform(df_test)
    return df_train, df_test
df_train, df_test = preprocessing(train, test)

[0.092, 0.331, 0.386, 0.365, 0.336, 0.454, 0.201, 0.3, 0.395, 0.145, 0.188, 0.225, 0.301, 0.252] ['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16']
Columns selected by correlation sum of the 3 first rows : 


,Selected columns,correlation total
0,measurement_8,0.454
1,measurement_11,0.395
2,measurement_5,0.386
3,measurement_6,0.365
4,measurement_7,0.336
5,measurement_4,0.331
6,measurement_15,0.301
7,measurement_10,0.300
8,measurement_16,0.252
9,measurement_14,0.225


False

-------- Product code A ----------

filled by linear model :
measurement_17 : 386
measurement_8 : 175
measurement_11 : 225
measurement_5 : 113
measurement_6 : 146
measurement_7 : 166
measurement_4 : 79
measurement_15 : 273
measurement_10 : 209
measurement_16 : 293
measurement_14 : 237

2302 filled by linear model 
1547 filled by KNN 

-------- Product code B ----------

filled by linear model :
measurement_17 : 418
measurement_8 : 165
measurement_11 : 220
measurement_5 : 90
measurement_6 : 106
measurement_7 : 176
measurement_4 : 80
measurement_15 : 294
measurement_10 : 197
measurement_16 : 358
measurement_14 : 330

2434 filled by linear model 
1541 filled by KNN 

-------- Product code C ----------

filled by linear model :
measurement_17 : 391
measurement_8 : 211
measurement_11 : 231
measurement_5 : 141
measurement_6 : 150
measurement_7 : 140
measurement_4 : 110
measurement_15 : 319
measurement_10 : 262
measurement_16 : 343
measurement_14 : 340

2638 filled by linear model 
170

In [66]:
df_train.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m3_missing,m4_missing,m5_missing,area,measurement_avg,measurement_std,measurement_median,measurement_max,measurement_min,measurement_skew
0,0,A,80.10,0.017894,material_8,9,5,7.0,8.0,4.0,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594000,15.193,15.029,15.495862,13.034,14.684,764.100,0.0,0,0,0,45,15.360919,2.776473,15.344431,20.155000,10.672,0.062682
1,1,A,84.89,0.017894,material_8,9,5,14.0,3.0,3.0,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915000,11.755,14.732,15.425000,14.395,15.631,682.057,0.0,0,0,0,45,15.446286,2.540705,15.528000,18.213000,11.540,-0.356926
2,2,A,82.43,0.017894,material_8,9,5,12.0,1.0,5.0,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,20.188481,13.798,16.711,18.631000,14.094,17.946,663.376,0.0,0,0,0,45,16.098820,2.671122,16.724500,20.188481,11.652,-0.312815
3,3,A,101.07,0.017894,material_8,9,5,13.0,2.0,6.0,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377000,10.020,15.250,15.562000,16.154,17.172,826.282,0.0,0,0,0,45,15.599500,2.933476,16.250000,19.060000,10.020,-0.705741
4,4,A,188.06,0.017894,material_8,9,5,9.0,2.0,8.0,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932000,12.428,16.182,12.760000,13.153,16.412,579.885,0.0,0,0,0,45,15.194071,3.027182,15.964000,19.932000,10.337,-0.035574


In [67]:
df_test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,m3_missing,m4_missing,m5_missing,area,measurement_avg,measurement_std,measurement_median,measurement_max,measurement_min,measurement_skew
0,26570,F,119.57,-0.07464,material_6,6,4,6.0,9.0,6.0,19.305,10.178,17.534,18.168,11.598,18.654,10.802,15.909,18.070000,13.772,13.659,16.825,13.742,17.710,634.612,NaN,0,0,0,24,15.423286,3.089647,16.3670,19.305,10.178,-0.480986
1,26571,F,113.51,-0.07464,material_6,6,4,11.0,8.0,0.0,17.883,11.927,17.228,16.033,11.179,19.368,12.032,13.998,17.958412,12.473,17.468,16.708,14.776,14.102,537.037,NaN,0,0,0,24,15.223815,2.654171,15.4045,19.368,11.179,-0.120501
2,26572,F,112.16,-0.07464,material_6,6,4,8.0,12.0,4.0,18.475,10.481,16.619,18.189,12.126,17.774,11.743,17.046,18.086000,10.907,13.363,15.737,17.065,16.021,658.995,NaN,0,0,0,24,15.259429,2.907555,16.3200,18.475,10.481,-0.588183
3,26573,F,112.72,-0.07464,material_6,6,4,8.0,11.0,10.0,16.518,10.888,15.293,18.592,11.304,18.948,11.790,18.165,16.163000,10.933,15.501,15.667,12.620,16.111,594.301,NaN,0,0,0,24,14.892357,2.870206,15.5840,18.948,10.888,-0.200273
4,26574,F,208.00,-0.07464,material_6,6,4,14.0,16.0,8.0,17.808,12.693,17.678,15.814,13.431,19.141,12.370,14.578,17.849000,11.941,16.070,16.183,13.324,17.150,801.044,NaN,0,0,0,24,15.430714,2.363162,15.9420,19.141,11.941,-0.094032


In [68]:
# features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 
#             'measurement_1', 'measurement_2','measurement_3','measurement_4',
#             'measurement_5', 'measurement_6','measurement_7','measurement_8',
#             'measurement_9','measurement_10','measurement_11', 'measurement_12',
#             'measurement_13','measurement_14','measurement_15','measurement_16', 'measurement_17',
#             'area', 'm3_missing', 'm5_missing', 'measurement_avg']
# 17 6 4 2 5miss 4miss 9 7 
# 17 0 1 2 3miss 5miss avg std median min skew
features = ['loading', 'attribute_0', 'measurement_17',
            'measurement_0', 'measurement_1', 'measurement_2',
            'area', 'm3_missing',
            'm4_missing', 'm5_missing', 'measurement_avg',
            'measurement_std', 'measurement_median', 'measurement_min',
            'measurement_skew']
features

['loading',
 'attribute_0',
 'measurement_17',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'area',
 'm3_missing',
 'm4_missing',
 'm5_missing',
 'measurement_avg',
 'measurement_std',
 'measurement_median',
 'measurement_min',
 'measurement_skew']

In [77]:
tf.keras.utils.set_random_seed(42)
test_predictions = np.zeros((df_test.shape[0], 1))

models = []
auc_score = []
targets = []
for fold in folds_dict.keys():
    
    print(f'########################## {fold} ##########################')
    
    x_train = df_train[df_train['product_code'].isin(folds_dict[fold][0])][features].values
    y_train = df_train[df_train['product_code'].isin(folds_dict[fold][0])]['failure'].values
    x_valid = df_train[df_train['product_code'].isin(folds_dict[fold][1])][features].values
    y_valid = df_train[df_train['product_code'].isin(folds_dict[fold][1])]['failure'].values
    trn_index = df_train[df_train['product_code'].isin(folds_dict[fold][0])][features].index
    vld_index = df_train[df_train['product_code'].isin(folds_dict[fold][1])][features].index
    
    print(f"x_train len:{len(x_train)} shape:{x_train.shape}")
    total_col = x_train.shape[1]
    model = Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(total_col*20, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(total_col*5, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(total_col, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(total_col-5, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation="sigmoid") 
    ])
    es_callbacks = tf.keras.callbacks.EarlyStopping(monitor="val_auc", 
                                                    patience = 7, restore_best_weights = True)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc",
                                                    factor=0.9, patience= 5, verbose=0, mode="max",
                                                    min_delta=0.0001, cooldown=0, min_lr=0)
    
    model.compile(optimizer = tfa.optimizers.AdamW(learning_rate = 1e-3, weight_decay = 1e-3 ), 
                  loss = "BinaryCrossentropy", metrics=["AUC"])
    model.fit(x_train,
              y_train,
              batch_size = 64,
              epochs = 200,
              callbacks = [es_callbacks, lr_scheduler],
              validation_data = (x_valid, y_valid)
              
             )
    predictions = model.predict(x_valid).reshape(-1, 1)
    test_pred = model.predict(df_test[features].values).reshape(-1, 1)
    models.append(model)
    
    score = roc_auc_score(y_valid, predictions)
    auc_score.append(score)
    test_predictions += test_pred * score
    
    print('\n')
    print(f'AUC score {fold} : {score}')
    print('\n')

test_predictions /= np.sum(np.array(auc_score))
models.append(auc_score)
with open('0711257_model.pkl', 'wb') as fp:
    pickle.dump(models, fp)
    print('Done saving models into a binary file')

print('\n')
print(f'CV AUC score: {np.mean(np.array(auc_score))}')

########################## Fold 1 ##########################
x_train len:16220 shape:(16220, 15)
Epoch 1/200
254/254 [==============================] - 5s 13ms/step - loss: 0.6477 - auc: 0.5248 - val_loss: 0.5604 - val_auc: 0.5404 - lr: 0.0010
Epoch 2/200
254/254 [==============================] - 3s 13ms/step - loss: 0.5485 - auc: 0.5226 - val_loss: 0.5296 - val_auc: 0.5600 - lr: 0.0010
Epoch 3/200
254/254 [==============================] - 2s 8ms/step - loss: 0.5265 - auc: 0.5476 - val_loss: 0.5210 - val_auc: 0.5662 - lr: 0.0010
Epoch 4/200
254/254 [==============================] - 2s 10ms/step - loss: 0.5186 - auc: 0.5591 - val_loss: 0.5178 - val_auc: 0.5866 - lr: 0.0010
Epoch 5/200
254/254 [==============================] - 2s 9ms/step - loss: 0.5178 - auc: 0.5508 - val_loss: 0.5197 - val_auc: 0.5781 - lr: 0.0010
Epoch 6/200
254/254 [==============================] - 2s 10ms/step - loss: 0.5154 - auc: 0.5664 - val_loss: 0.5154 - val_auc: 0.5851 - lr: 0.0010
Epoch 7/200
254/254 [==

In [78]:
# submission['failure'] = test_predictions
# submission[["id", "failure"]].to_csv('0711257_submission.csv', index=False)
# submission.to_csv('0711257_submission.csv', index=False)

In [71]:
# tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=100, expand_nested=True)

In [72]:
# pip freeze > requirements.txt